In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import lit
spark = SparkSession.builder.master("local[*]").appName('').getOrCreate()

In [ ]:
df = spark.read.parquet("./intermunicipal_sc/ref_join/parquet/singlefile/")

In [ ]:
df.createOrReplaceTempView("df")

In [ ]:
print(f"{df.count()} linhas | {len(df.columns)} colunas")

In [ ]:
df.printSchema()

In [ ]:
sql = spark.sql("SELECT ano, empresa, modalidade, linha, sig_secao_ori_des AS secao, km, passageiros_total AS pax_total FROM df").show(20, False)

In [ ]:
import pandas as pd
dfp = pd.read_parquet("./intermunicipal_sc/ref_join/parquet/singlefile")
dfp.head()

# ANTT

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import lit
spark = SparkSession.builder.master("local[*]").appName('').getOrCreate()

antt = spark.read.option("header", "True").parquet("./interestadual/ref/parquet/")
antt.createOrReplaceTempView("antt")
# antt.printSchema()

In [ ]:
empresa = ""
origem = "CHAPECO-SC"
destino = origem
ano = "2019"

df_temp = spark.sql(f"""

SELECT '{origem}' AS cidade_base, ano, mes, empresa, prefixo, ori_des_localidade_nome, pax_total
FROM antt
WHERE pax_total > 0
AND ano = {ano}
AND empresa LIKE '%{empresa}%'
AND (ori_des_localidade_nome LIKE '%{origem}%' OR ori_des_localidade_nome LIKE '%{destino}%')
ORDER BY ano ASC, empresa ASC, prefixo ASC, pax_total DESC

""")

if empresa == "":
    if origem == destino:
        df_temp.toPandas().to_excel(f"./interestadual_filtro/{ano}_{origem}.xlsx", index = False)
    else:
        df_temp.toPandas().to_excel(f"./interestadual_filtro/{ano}_{origem}_{destino}.xlsx", index = False)
    
else:
    if origem == destino:
        df_temp.toPandas().to_excel(f"./interestadual_filtro/{ano}_{empresa}_{origem}.xlsx", index = False)
    else:
        df_temp.toPandas().to_excel(f"./interestadual_filtro/{ano}_{empresa}_{origem}_{destino}.xlsx", index = False)
    
# if empresa == "":
#     df_temp.withColumn("cidade_base", lit(origem)).toPandas().to_excel(f"{ano}_{origem}_{destino}.xlsx", index = False)
# else:
#     df_temp.withColumn("cidade_base", lit(origem)).toPandas().to_excel(f"{ano}_{empresa}_{origem}_{destino}.xlsx", index = False)    